# Age recognition 

**Authors**: Richard Šléher, Tomáš Majerník

**Dataset**: https://www.kaggle.com/datasets/arashnic/faces-age-detection-dataset/code?select=train.csv

# TODO
- natrenovat
- pripravit valid/test dataset 
- wandb
- pozret sa ci treba zmenit weighty ked mame pre nejaku classu viac obrazkov aby castejsie classificovalo

In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from summarytools import dfSummary
import random
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image, ImageOps
import torch.optim as optim
import torch.nn as nn

Hyperparameters

In [ ]:
IMAGE_SIZE = 224
epoch = 30
batch_size = 32
lr = 0.1e-4

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## EDA

In [ ]:
data = pd.read_csv('data/train.csv')

In [ ]:
dfSummary(data)

In [ ]:
train_df, temp_df = train_test_split(data, train_size=0.8, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=True, random_state=42)

In [ ]:
dfSummary(train_df)

In [ ]:
dfSummary(valid_df)

In [ ]:
dfSummary(test_df)

In [ ]:
fig = plt.figure()

for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread('data/train/' + train_df.iloc[i]['ID'])
    plt.imshow(img)
    plt.title(train_df.iloc[i]['Class'])
    plt.axis('off')

plt.show()

In [ ]:
def load_data(dataframe: pd.DataFrame, directory: str, transform: transforms.Compose):
    if not os.path.exists(directory):
        os.makedirs(directory)

    images = []
    labels = []
    label_mapping = {'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2}
    augmented_dir = os.path.join(directory, 'Augmented')

    for idx in range(len(dataframe)):
        img_name = os.path.join(directory, dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = dataframe.iloc[idx, 1]
        if transform:
            transformed_image = transform(image)
        images.append(transformed_image)
        labels.append(label_mapping[label])

    return torch.stack(images).to(device), torch.tensor(labels).to(device)

In [ ]:
def calculate_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    mean = 0.0
    std = 0.0
    total_images_count = 0
    
    for images, _ in loader:
        images = images.view(batch_size, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_size
    
    mean /= total_images_count
    std /= total_images_count
    
    return mean, std

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

valid_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_images, train_labels = load_data(train_df, 'data/Train/', train_transform)
#valid_images, valid_labels = load_data(valid_df, 'data/Test/', valid_test_transform)
#test_images, test_labels = load_data(test_df, 'data/Test/', valid_test_transform)

train_loader = [(train_images[i:i + batch_size], train_labels[i:i + batch_size]) for i in range(0, len(train_images), batch_size)]
#valid_loader = [(valid_images[i:i + batch_size], valid_labels[i:i + batch_size]) for i in range(0, len(valid_images), batch_size)]
#test_loader = [(test_images[i:i + batch_size], test_labels[i:i + batch_size]) for i in range(0, len(test_images), batch_size)]

# Shape of the data for DL model
for images, labels in train_loader:
    print(images.shape, len(labels))
    break


In [ ]:
for images, labels in train_loader:
    print(labels)
    break

## Model

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes: int):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
# Example usage:
model = VGG16(num_classes=3)
print(model)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(epoch):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epoch}, Loss: {running_loss/len(train_loader)}")

print("Training finished.")